In [1]:
import kenlm
import spacy
import os

In [2]:
model = kenlm.LanguageModel(r"C:\Users\Louis\kenlm\lm\test.arpa")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "kenlm.pyx", line 139, in kenlm.Model.__init__
RuntimeError: C:\Users\Louis\AppData\Local\Temp\pip-install-j2jo494r\kenlm_910c9b2d4ec142fe89ef7481639df3a5\lm\read_arpa.hh:84 in lm::ReadNGram threw FormatLoadException because `index == 0 && (word != StringPiece("<unk>", 5)) && (word != StringPiece("<UNK>", 5))'.
Word good was not seen in the unigrams (which are supposed to list the entire vocabulary) but appears in the 3-gram at byte 2489 Byte: 2489

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Louis\AppData\Local\Temp\ipykernel_23728\927024240.py", line 1, in <module>
    model = kenlm.LanguageModel(r"C:\Users\Loui

Hello